# Generowanie i Analiza Biometrii użytkownika poprzez identyfikację naciśnięć klawiszy
## 1. Założenia:
### a. Czym jest biometria wciśnięć klawiszy
Każdy człowiek posiada unikalną strukturę pisania. Poprzez mikro różnicę takie jak długość palcy, sposób nauki pisania czy wzorce behawioralne wyrabiamy sposób pisania unikalny dla siebie. Przy odpowiednio dużej próbie można namierzyć każdego.
### b. Jak identyfikować użytkownika
Zbierane są kluczowe dane:
- jaki klawisz był naciśnięty (key)
- kiedy był wciśnięty (press_time)
- kiedy puszczony (release_time)
- ile był przytrzymany (dwell_time) -> (release_time - press_time)
- ile trwało przejście pomiędzy klawiszami (flight_time) (press_time_key_b - release_time_key_A)
- unikatowe id sesji użytkownika potrzebne do jego identyfikacji wstępnej (treningu)

Te dane zbierane sa dla ciągu znaków i zapisywane do bazy danych po czym analizowane przez backend. W zależności od metody identyfikacja może być bardziej dokładna ale też kosztowniejsza w obliczeniach. W moim skryptcie użyłem średnich jako że są najszybszą i najmniej wymagającą obliczeniowo operacją potrzebną do zidentyfikowania.



In [ ]:
class KeystrokeData(BaseModel):
    dwellTime: int
    flightTime: int
    key: str
    keyCode: str
    pressTime: int
    releaseTime: int

Profil użytkownika jest wyliczany poprzez wyciągnięcie dystansu manhatanskiego i odjecie jego wyniku od 100 z czego zostaje nam pewność na ile dany użytkownik jest autorem tekstu

In [ ]:
dist = abs(decimal.Decimal(in_dwell) - user_dwell) + abs(decimal.Decimal(in_flight) - user_flight)

confidence = max(0, 100 - dist)

API zostało zrobione za pomocą FastAPI (https://fastapi.tiangolo.com/) wraz z backendem w postgresql^17 postawionym na dockerze. Użyto dotenv dla zabezpieczenia danych wrażliwych. Niestety nie znalazłem odpowiednich danych do nauczenia maszynowego a wszelkie próby na próbach 2 osób nie pozwoliły mi na dokładne działanie. Próbowałem wykorzysytać (https://www.cs.cmu.edu/~keystroke/) jednak próby z większym tekstem czy językiem polskim wchodziły do błędu statystycznego i nie dawały miarodajnego wyniku. Po kilku próbach najstabliniejszy okazał się algorytm odległości Manhatanskiej i pomimo średniej celnośći próby na kilku (2-3) osobach okazły się wykrywać z dokładnością między 75-90%. Dodałem opcję "przetrenowania" oraz wykrycia podszywania się pod inne osoby podczas treningu. Dane zbierane są przez frontend napisany w react+vite (wyprany głownie przez hot swap modułów co pozwoliło na szybki mockup i testy). Dołączę zbudowany front który można postawić poprzez server apache lub node pamiętając o CORS. Backend stoi na porcie 8000 a poprzez użycie FastApi jest dostępny Swagger pod adresem lokalnym z suffixem /docs np. http://localhost:8000/docs
zamiast pip użyłem uv https://github.com/astral-sh/uv głównie przez jego szykość i możliwość użycia narzędzi bez potrzeby instalacji ich do globalnego środowiska za pomocą uv tool. Dla pewności przesyłam plik requrements.txt wygenerowany za pomocą komendy
```
uv pip freeze > requirements.txt
```
